In [ ]:
#!Begin

# Masters' in Applied Artificial Intelligence
## Machine Learning Algorithms Course

Notebooks for the MLA course

by [*lufer*](mailto:lufer@ipca.pt)

(ver 2.0)

---



# ML Modelling - Part IV - Deploying Models

**Contents**:

1. **Deploying Models**
2. **Prepare model to be deployed**




This notebook explainh how to deploy a Machine Learning model

# Deploying Models

# Environment preparation

1. Model preparation
2. Project Environment


## Build the machine learning model

This is a very simple demonstrative model creation. This code loads the California housing dataset, selects three features (MedInc, AveRooms, AveOccup), trains a linear regression model, and saves it in the model/ folder of your local drive. The saved model has name *linearRegressionModel.pkl*

**Install necessary Libraries**

In [ ]:
# model_training.py
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import os

In [ ]:
import datetime
print(f"Last updated: {datetime.datetime.now()}")

Last updated: 2025-02-16 13:34:06.982938


**Mounting Drive**

In [ ]:

from google.colab import drive

# it will ask for your google drive credentiaals
drive.mount('/content/gDrive/', force_remount=True)

Mounted at /content/gDrive/


### Getting data

In [ ]:
# Load the dataset
data = fetch_california_housing(as_frame=True)
df = data['data']
target = data['target']

In [ ]:
#check dataset size
df.shape

(20640, 8)

### Get Features and Target

In [ ]:
# Select only some features of the entire dataset
selected_features = ['MedInc', 'AveRooms', 'AveOccup']
X = df[selected_features]
y = target

### Train the model

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### Check Prediction accuracy

In [ ]:
#check the model prediction
#all X_test
model.predict(X_test)

array([1.15826883, 1.4999938 , 1.96250199, ..., 4.33715353, 1.60108676,
       1.9844191 ])

In [ ]:
#check only the first instance of X_test
print(X_test.values[0])
priceHouse = model.predict([X_test.values[0]])
priceHouse

[1.6812     4.19220056 3.87743733]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([1.15826883])

In [ ]:
#check any other input values
#create array of values
inputs=[3.5,5.0,2.0]
priceHouse = model.predict([inputs])
print(inputs)
priceHouse


[3.5, 5.0, 2.0]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([1.92739533])

In [ ]:
#check manually created instance (with feature names)
manual_instance = {
    "MedInc": 5.3,
    "AveRooms": 7.4,
    "AveOccup": 1.6
}

# Convert it to a format that your model can accept (a list of lists)
#inputs = [manual_instance["MedInc"], manual_instance["AveRooms"], manual_instance["AveOccup"]]
#priceHouse = model.predict(inputs)

#or

# Convert the instance to a pandas DataFrame with the correct feature names
inputs_df = pd.DataFrame([manual_instance])
priceHouse = model.predict(inputs_df)
print(inputs)
priceHouse

[3.5, 5.0, 2.0]


array([2.61828368])

### Exporting the model

In [ ]:
# Create a 'model' folder to save the trained model
#os.makedirs('model', exist_ok=True)
#or
modelPath="/content/gDrive/MyDrive/Colab Notebooks/MIA - ML - 2024-2025/Model/"

In [ ]:
# Save the trained model using pickle
with open(modelPath+'linearRegressionModel.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model trained and saved successfully.")

Model trained and saved successfully.


In [ ]:
#Export the X_test dataset
# Save X_test to a CSV file
X_test.to_csv(modelPath+"X_test.csv", index=False)
print("X_test saved successfully.")

X_test saved successfully.


## Deploy and Use the ML model

There are many different approaches to deploy and work with a machine learning model. In this approach, two things are required:

1. Know how to work with docker containners
2. Know how to work with Services API


Considering Docker Containers:

1. Have Python installed (>=3.11)
2. Have Docker installed (for your OS)


1 - Create a Python Virtual Environment

In [ ]:
#create
python -m venv v1
#activate it
v1\Scripts\activate
#to return to the normal python (get out the venv)
#deactivate

#in linux
#source v1/bin/activate

2-Install the required packages


```
pip3 install pandas scikit-learn fastapi uvicorn
```

3-Train and Export the Model


```
python3 model_training.py
```


4 - Creating the FastAPI App

Inside the app/ folder, create two files: **__init__.py** (empty) and **server.py**.

**server.py** should have:

```
# app/server.py
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import os

# Define the input data schema using Pydantic
class InputData(BaseModel):
    MedInc: float
    AveRooms: float
    AveOccup: float

# Initialize FastAPI app
app = FastAPI(title="House Price Prediction API")

# Load the model during startup
model_path = os.path.join("model", "linearRegressionModel.pkl")
with open(model_path, 'rb') as f:
    model = pickle.load(f)

@app.post("/predict")
def predict(data: InputData):
    # Prepare the data for prediction
    input_features = [[data.MedInc, data.AveRooms, data.AveOccup]]
    
    # Make prediction using the loaded model
    prediction = model.predict(input_features)
    
    # Return the prediction result
    return {"predicted_house_price": prediction[0]}
```



5 - Containerizing the App with Docker

1. Create the "Dockerfile" - all containerization steps
2. CReate the "requirements" file - all required packages to include in the docker

*Dockerfile*

```
FROM python:3.11-slim

# Set the working directory inside the container
WORKDIR /code

# Copy the requirements file
COPY ./requirements.txt /code/requirements.txt

# Install the Python dependencies
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt

# Copy the app folder into the container
COPY ./app /code/app

# Copy the model directory (with the saved model file) into the container
COPY ./model /code/model

# Expose port 8000 for FastAPI
EXPOSE 8000

# Command to run the FastAPI app with Uvicorn
CMD ["uvicorn", "app.server:app", "--host", "0.0.0.0", "--port", "8000"]
```

*requirementx.txt*

```
fastapi
uvicorn
scikit-learn
pandas
```

6 - Building the Docker Image

you must have docker manager running...

```
docker build -t prediction-api .
```

7 - Run the docker container

```
docker run -d --name miaa-test -p 8000:8000 prediction-api
```

8 - When changing FastAPI

If FastAPI is changed, the docker must be rebuild

1. Remover the current docker

```
docker ps  # Check running containers
docker stop <container_id>  # Stop the running container
docker rm <container_id>  # Remove the stopped container
```
in our case the *container-id="miaa-test"*

2. Rebuild the Docker Image
  * *docker build -t prediction-api .*

3. Run the docker
  * *docker run -d --name miaa-test1 -p 5000:8000 prediction-api*


9 - Ckeck the FastAPI

1. With Postman

* POST; http://127.0.0.1:8000/predict; define Body Raw input data

```
{
    "MedInc": "3.5",
  "AveRooms": "5.0",
  "AveOccup": "2.0"
}
```

* POST; http://127.0.0.1:8000/predictMany; define Body Raw input data

```
{
  "instances": [
    { "MedInc": 8.3252, "AveRooms": 6.984127, "AveOccup": 2.555556 },
    { "MedInc": 7.2574, "AveRooms": 8.288136, "AveOccup": 2.802260 }
  ]
}
```

2. By a *curl* command

* define json data into a *input.json* file
```
{
  "MedInc": 3.5,
  "AveRooms": 5.0,
  "AveOccup": 2.0
}
```
* set curl command
```
curl -X POST -H "Content-Type: application/json" -d @input.json http://localhost:8000/predict
```

* define json data into a *inputMany.json* file
```
{
  "instances": [
    { "MedInc": 8.3252, "AveRooms": 6.984127, "AveOccup": 2.555556 },
    { "MedInc": 7.2574, "AveRooms": 8.288136, "AveOccup": 2.802260 }
  ]
}
```
* set curl command
```
curl -X POST -H "Content-Type: application/json" -d @inputMany.json http://localhost:8000/predictMany
```


3. By Flask+HTML

* install flask

```
pit install flask
```
* create flask python file *test.py* and *templates/index.html*

* run flask files: *python test.py*

* check http://127.0.0.1:5001




10 - Debug the FastAPI

```
uvicorn app.server:app --host 0.0.0.0 --port 8000 --reload
```

Summary


```
.
├── Dockerfile
├── requirements.txt
├── app/
│   └── server.py  # the FastAPI code
├── model/
│   └── linearRegressionModel.pkl  # the pickle model

```



# References

*  [Explainable AI - Understanding and Trusting Machine Learning Models](https://www.datacamp.com/tutorial/explainable-ai-understanding-and-trusting-machine-learning-models)  
*  [Why is explainability important?](https://xai-tutorials.readthedocs.io/en/latest/_xai/importance.html)
*  [SHapley Additive exPlanations (SHAP)](https://xai-tutorials.readthedocs.io/en/latest/_model_agnostic_xai/shap.html)
*  [Local Interpretable Model-Agnostic Explanations (LIME)](https://xai-tutorials.readthedocs.io/en/latest/_model_agnostic_xai/lime.html)
*  [Eli5 (Explain it like I am 5) Model Explainability in Python](https://medium.com/chat-gpt-now-writes-all-my-articles/eli5-explain-it-like-i-am-5-model-explainability-in-python-d4922f021037)
*  [ELI5’s documentation!](https://eli5.readthedocs.io/en/latest/overview.html)
*  [Techniques for Interpreting and Explaining ML Models](https://www.markovml.com/blog/model-interpreting)

Applied

*  [Explanatory Model Analysis (Section 3.2.2)](https://ema.drwhy.ai/)